In [ ]:
#*****************************
# nome: Paria
# cognome: Haji Abolfath
# 2044012
#*****************************

!pip install torch_geometric
# !pip3 install datasets

In [2]:
import torch
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader 
# functionals ?? loss funtion
import torch.nn.functional as F

import pandas as pd
from torch_geometric.data import Data
from decimal import Decimal
from torch.utils.data import RandomSampler, DataLoader, Subset
from torch_geometric.transforms import BaseTransform, Compose
import os.path

In [3]:
dataAlias = 'DHFR'
dataset = TUDataset(root="/".join(['.','tmp']), name=dataAlias)

# shuffle the whole dataset - IS MOVED to several cells below, because of additional data pre-processing
# perm = torch.randperm(len(dataset))
# dataset = dataset[perm]

Extracting tmp/DHFR/DHFR.zip
Processing...
Done!


In [4]:
cte_shape_for_all = ((dataset[0].x).shape)[1] # 53 - one-hot representation of the chemical atom
var_shape_among_all = ((dataset[0].x).shape)[0] # the number of nodes in the graph
# print(var_shape_among_all)

# so we have 53 atoms -
# each is represented by a binary num - and thus can be represented by a single integer as well
a=(dataset[0]).x
# print(a.shape[0])
num_of_atoms = 0
num_of_tot_atoms = 53
graph_int_ = "" # an integer representing the atoms in the graph
c=0

graph_node_count=[]
for i in range(len(dataset)):
  for j in range(((dataset[i].x).shape)[0]): # iterate over the number of the atoms in graph i
      graph_int_ = graph_int_+str(int(((torch.nonzero((dataset[i].x)[j], as_tuple=True))[0])[0]))
      c=c+1
  graph_node_count.append(c)
  c=0
  graph_int_ = "" # reset for the next graph (idx i)

graph_edge_count=[]
c=0
for i in range(len(dataset)):
  for j in range((((dataset[i]).edge_index).shape)[1]):
    c=c+1
  graph_edge_count.append(c)
  c=0

30
30
7
(tensor([7]),)
tensor([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
torch.Size([32075, 53])


###### each element is a graph. And the descriptive info that comes with it contains:
###### 1. edge_index : describes the connections between nodes (edges) , always of dim (2, num_edges); 2 because of bi-directional
###### 2. x is a one-hot representation of the nodes in the graph (Node feature matrix with shape [num_nodes, num_node_features])
######     ( in the case where we have no features intially, this tensors is majorly filled with 0s, representing the nodes from 0 to num_nodes-1)
###### 3. y label of the graph

#####https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.data.Data.html#torch_geometric.data.Data

In [7]:
# degree
df = open('texts/DegOut_.txt', "r")
lines_d = df.readlines()
df.close()

# remove /n at the end of each line
for index, line in enumerate(lines_d):
      lines_d[index] = line.strip()

# betweenness centrality
df = open('texts/BetOut_.txt', "r")
lines_b = df.readlines()
df.close()

for index, line in enumerate(lines_b):
      lines_b[index] = line.strip()

# closeness centrality
df = open('texts/ClosOut_.txt', "r")
lines_c = df.readlines()
df.close()

for index, line in enumerate(lines_c):
      lines_c[index] = line.strip()

# graphlets of length three : triangles
df = open('texts/graphlet_counts_len_3.txt', "r")
lines_g3 = df.readlines()
df.close()

for index, line in enumerate(lines_g3):
      lines_g3[index] = line.strip()

# graphlets of length four : ??
df = open('texts/graphlet_counts_len_4.txt', "r")
lines_g4 = df.readlines()
df.close()

for index, line in enumerate(lines_g4):
      lines_g4[index] = line.strip()

In [8]:
# convert the features from string to decimal

# betweenness centrality
lines_b_m = lines_b # 'm' for modified
for i in range (len(lines_b)):
  lines_b_m[i] = (Decimal(lines_b[i]))

# degree
lines_d_m = lines_d # 'm' for modified
for i in range (len(lines_d)):
  lines_d_m[i] = (Decimal(lines_d[i]))

# closeness centrality
lines_c_m = lines_c # 'm' for modified
for i in range (len(lines_c)):
  lines_c_m[i] = (Decimal(lines_c[i]))

# graphlet len 4
lines_g4_m = lines_g4 # 'm' for modified
for i in range (len(lines_g4)):
  lines_g4_m[i] = (Decimal(lines_g4[i]))

# graphlet len 3
lines_g3_m = lines_g3 # 'm' for modified
for i in range (len(lines_g3)):
  lines_g3_m[i] = (Decimal(lines_g3[i]))

In [9]:
# each graph will have 3 new features : avg degree, avg bet, avg clos

# *** DEGREE DEGREE ***
deg_sum = 0
deg_avg_all = []
deg_lines_idx = 0

for i in range (len(dataset)): # 756 - num of graphs
  for j in range((dataset[i]).num_nodes):
    deg_sum = deg_sum + lines_d_m[deg_lines_idx]
    deg_lines_idx = deg_lines_idx + 1
  deg_avg_all.append((deg_sum/(dataset[i]).num_nodes))
  deg_sum = 0

# *** VARIANCE OF DEGREE *** : (sum(lines_d_m[deg_lines_idx] - deg_avg_all[i])) / ((dataset[i]).num_nodes)
sum_of_diffs = 0
var_deg_all = []
deg_lines_idx = 0

for i in range (len(dataset)):
  for j in range((dataset[i]).num_nodes):
    sum_of_diffs = sum_of_diffs + abs(lines_d_m[deg_lines_idx] - deg_avg_all[i])
    deg_lines_idx = deg_lines_idx + 1
  var_deg_all.append(((sum_of_diffs) ** 2) / ((dataset[i]).num_nodes) - 1)
  sum_of_diffs = 0

In [10]:
# *** AVERAGE BETWEENNESS CENTRALITY ***
bet_sum = 0
bet_avg_all = []
bet_lines_idx = 0

for i in range (len(dataset)):
  for j in range((dataset[i]).num_nodes):
    bet_sum = bet_sum + lines_b_m[bet_lines_idx]
    bet_lines_idx = bet_lines_idx + 1
  bet_avg_all.append((bet_sum/(dataset[i]).num_nodes))
  bet_sum = 0

# *** VARIANCE OF BETWEENNESS CENTRALITY ***
sum_of_diffs = 0
var_bet_all = []
bet_lines_idx = 0

for i in range (len(dataset)):
  for j in range((dataset[i]).num_nodes):
    sum_of_diffs = sum_of_diffs + abs(lines_b_m[bet_lines_idx] - bet_avg_all[i])
    bet_lines_idx = bet_lines_idx + 1
  var_bet_all.append(((sum_of_diffs) ** 2) / ((dataset[i]).num_nodes) - 1)
  sum_of_diffs = 0

In [11]:
# *** AVERAGE CLOSENESS CENTRALITY ***
close_sum = 0
close_avg_all = []
close_lines_idx = 0

for i in range (len(dataset)):
  for j in range((dataset[i]).num_nodes):
    close_sum = close_sum + lines_c_m[close_lines_idx]
    close_lines_idx = close_lines_idx + 1
  close_avg_all.append((close_sum/(dataset[i]).num_nodes))
  close_sum = 0

# *** VARIANCE OF CLOSENESS CENTRALITY ***
sum_of_diffs = 0
var_close_all = []
close_lines_idx = 0

for i in range (len(dataset)):
  for j in range((dataset[i]).num_nodes):
    sum_of_diffs = sum_of_diffs + abs(lines_c_m[close_lines_idx] - close_avg_all[i])
    close_lines_idx = close_lines_idx + 1
  var_close_all.append(((sum_of_diffs) ** 2) / ((dataset[i]).num_nodes) - 1)
  sum_of_diffs = 0

In [12]:
# now we have 3 lists for the averages, those need to be converted to tensors first - that's requierd in order to work with the DL methods

# betweennesss centrality
torch.set_printoptions(precision=17)
torch.set_default_dtype(torch.float32)
betweenness_tensor = torch.Tensor(var_bet_all)

# closeness centrality - seems to be more useful in discriminating between different graphs
torch.set_printoptions(precision=17)
torch.set_default_dtype(torch.float32)
closeness_tensor = torch.Tensor(var_close_all)

# degree
torch.set_printoptions(precision=17)
torch.set_default_dtype(torch.float32)
degree_tensor = torch.Tensor(var_deg_all)

# graph node count
node_count_tensor = torch.Tensor(graph_node_count)
# graph edge count
edge_count_tensor = torch.Tensor(graph_edge_count)

# graphlet length 3
graphlet_three_tensor = torch.Tensor(lines_g3_m)

# graphlet length 4
graphlet_four_tensor = torch.Tensor(lines_g4_m)

In [ ]:
# logistic regression - solve the issue with one-hot in x - variance - DONE
# TUDataset is a Homogeneous dataset - for which it is used a Data which is : A data object describing a homogeneous graph.
# a homogeneous dataset contains data of a unique type for instance all are numerical
# but in a heterogeneous one, we have data of types numerical, categorical, textual e cosi via.

# in HDFR, we have only numerical information so it's a homogeneous one.

<class 'torch_geometric.datasets.tu_dataset.TUDataset'>


In [ ]:
# save an object to a disk file, tensors and models are saved used the .pt file format

# THIS BLOCK IS EXECUTED ONLY ONCE (FROM FOLLOWING EXECUTATIONS, THE TENSORS WILL ONLY BE LOADED)
torch.save(degree_tensor, 'tensors/degrees.pt')
torch.save(betweenness_tensor, 'tensors/betweenness_centrality.pt')
torch.save(closeness_tensor, 'tensors/closeness_centrality.pt')
torch.save(graphlet_four_tensor, 'tensors/graphlet_four.pt')
torch.save(graphlet_three_tensor, 'tensors/graphlet_three.pt')

torch.save(graph_edge_count, 'tensors/edge_count.pt')
torch.save(graph_node_count, 'tensors/node_count.pt')

In [13]:
degrees_t = torch.load('tensors/degrees.pt')
closeness_t = torch.load('tensors/closeness_centrality.pt')
betweenness_t = torch.load('tensors/betweenness_centrality.pt')
graph_three_t = torch.load('tensors/graphlet_three.pt')
graph_four_t = torch.load('tensors/graphlet_four.pt')

nodes_c_t = torch.load('tensors/node_count.pt')
edges_c_t = torch.load('tensors/edge_count.pt')

torch.set_printoptions(precision=17)
torch.set_default_dtype(torch.float64)

degrees_t = torch.Tensor(degrees_t)
closeness_t = torch.Tensor(closeness_t)
betweenness_t = torch.Tensor(betweenness_t)
nodes_c_t = torch.Tensor(nodes_c_t)
edges_c_t = torch.Tensor(edges_c_t)
graph_three_t = torch.Tensor(graph_three_t)
graph_four_t = torch.Tensor(graph_four_t)

In [14]:
# fist stack the features together
tensor_new = torch.column_stack((degrees_t, closeness_t, betweenness_t, nodes_c_t, edges_c_t, graph_three_t, graph_four_t))

# A data object describing a homogeneous graph
new_dataset = Data(x=tensor_new, y=dataset.y)

In [15]:
# shuffle the whole dataset - this is done after the incorporation is completed

# perm = torch.randperm(756) # a tensor containing random numbers
# sample_y=torch.index_select(new_dataset.y, 0, perm)
# sample_x=torch.index_select(new_dataset.x, 0, perm) # take the corrsponding slice from labels
train_fraction = int((756)*0.7)

train_x = (new_dataset.x)[:train_fraction]
train_y = (new_dataset.y)[:train_fraction]
train_y  = train_y.to(torch.float64)
# train_dataset = Data(x=train_sliced_permutated_x, y=train_sliced_permutated_y)

test_x = (new_dataset.x)[train_fraction:]
test_y = (new_dataset.y)[train_fraction:]
test_y  = test_y.to(torch.float64)
# test_dataset = Data(x=test_sliced_permutation_x, y=test_sliced_permutation_y)

In [16]:
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __getitem__(self, index):
        x = self.features[index]
        y = self.labels[index]
        return {'x': x, 'y': y}

    def __len__(self):
        return len(self.features)

In [92]:
# standarization on the features (SCALING)
from sklearn.preprocessing import StandardScaler
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

scaler = StandardScaler()

# TRAIN
X_train_standardized = torch.Tensor(scaler.fit_transform(train_x))

train_ds = CustomDataset(X_train_standardized, train_y)
batch_size = 16
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=False)

for batch in train_loader:
    features = batch['x'].to(device)
    labels = batch['y'].to(device)

# TEST
X_test_standardized = torch.Tensor(scaler.fit_transform(test_x))
test_ds = CustomDataset(X_test_standardized, test_y)
test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False)

for batch in test_loader:
    features = batch['x'].to(device)
    labels = batch['y'].to(device)

In [18]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_size):
        super(LogisticRegression, self).__init__()
        self.fc1 = torch.nn.Linear(input_size,1)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.sigmoid(x)
        return x

In [93]:
# input_size = (train_x.shape)[1] # or maybe just 64 - batch size
input_size = 7 # num of features
model = LogisticRegression(input_size)
model.apply(lambda m: torch.nn.init.xavier_uniform_(m.weight) if isinstance(m, torch.nn.Linear) else None)
learning_rate = 0.5
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
criterion = torch.nn.CrossEntropyLoss()

In [20]:
def train(data_loader):
  for batch in data_loader:
      features = batch['x']
      labels = batch['y']

      # Forward pass
      optimizer.zero_grad()

      probabilities = model(features)
      # probabilities = probabilities.squeeze(1)  # Convert y from shape [64] to [64, 1]
      # print(probabilities)
      # Compute loss
      probabilities = probabilities.squeeze(1)  # Convert y from shape [64] to [64, 1]
      # print(probabilities)

      loss = criterion(probabilities, labels)

      # Backward pass and optimization
      loss.backward()

      # for name, param in model.named_parameters():
      #   if param.grad is not None:
      #       print(f'Parameter: {name}, Gradient Norm: {param.grad.norm().item()}')
      optimizer.step()

In [66]:
def test_eval(loader):
    model.eval()
    correct = 0
    total = 0
    accuracy_lst = []
    t_lst = []
    prob_lst=[]
    for data in loader:  # Iterate in batches over the training/test dataset.
        out = model(data['x'])
        labels = data['y']

        threshold = 0.53
        predicted = (out > threshold).float()
        predicted = predicted.squeeze(1)
        prob_lst.append(out)
        t_lst.append(labels)

        correct = torch.sum((predicted == labels).float()).item()
        total = labels.size(0)

        accuracy = 100 * correct / total
        accuracy_lst.append(accuracy)
    return accuracy_lst, prob_lst, t_lst

In [94]:
for epoch in range(100):
    train(train_loader)
    test_acc, p, t = test_eval(test_loader)

In [95]:
from statistics import mean
print(mean(test_acc))

74.58333333333333


In [71]:
print(max(test_acc))

93.75


In [96]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris

train_ds = CustomDataset(X_train_standardized, train_y)
test_ds = CustomDataset(X_test_standardized, test_y)
tree_classifier = DecisionTreeClassifier()

tree_classifier.fit(X_train_standardized, train_y)

y_pred = tree_classifier.predict(X_train_standardized)

accuracy = accuracy_score(train_y, y_pred)
print(f'Accuracy of train: {accuracy}')

# Make predictions on the test set
y_pred = tree_classifier.predict(X_test_standardized)

# Evaluate the accuracy
accuracy = accuracy_score(test_y, y_pred)
print(f'Accuracy of test: {accuracy}')


Accuracy of train: 0.9640831758034026
Accuracy of test: 0.6431718061674009
